#

# LangGraph Detailed Agent
This advances the agent that was done in the langgraph_agent.ipynb notebook and focuses on defining specific roles to the agents/nodes.  Review that one first to see details on the steps as this one will only explain the additional details.

### 1. Load the needed libraries and environment variables

In [ ]:
from dotenv import load_dotenv
from typing import List, Any, Optional, Annotated, TypedDict
from pydantic import BaseModel, Field

import os
import sqlite3
import gradio 

from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import BaseTool
from langsmith import uuid7
from langchain_community.tools import Tool
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.agent_toolkits import FileManagementToolkit
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import Image

In [ ]:
load_dotenv(override=True)

### 2. Add Tools

In [ ]:
def get_tools() -> List[BaseTool]:
    # Define server, wrapper around the Google API, as a tool
    tool_search: Tool = Tool(
        name="search",
        func=GoogleSerperAPIWrapper().run,
        description="Use this tool when you want to get the results of an online web search"
    )

    # Define Wikipedia as a search as a tool
    wikipedia: WikipediaAPIWrapper = WikipediaAPIWrapper()
    wiki_tool: WikipediaQueryRun = WikipediaQueryRun(api_wrapper=wikipedia)

    # Define file management tools
    file_tools: List[BaseTool] = FileManagementToolkit(root_dir="sandbox").get_tools()

    # Concatenate all tools together as a list
    tools: List[BaseTool] = file_tools + [tool_search, wiki_tool]

    return tools

### 3. Setup Memory

In [ ]:
db_path = "memory.db"
conn = sqlite3.connect(db_path, check_same_thread=False)
sql_memory = SqliteSaver(conn)

### 4. Build the Graph
It is now time to put start building the graph.  All graphs include the following steps:
<ol>
<li>Define the State</li>
<li>Start Graph Builder</li>
<li>Create Nodes</li>
<li>Create Edges</li>
<li>Compile the Graph</li>
</ol>

### State with additional details
Will track additional details in the state to help with routing:
<ul>
<li>feedback_on_work: details from the evaluator</li>
<li>success_criteria_met: determines if it should end</li>
<li>number_of_reviews: the number of reviews completed as a guardrail from looping too much</li>
</ul>

In [ ]:
# Step 1: Define the State object
class State(TypedDict):
    messages: Annotated[List[Any], add_messages]
    feedback_on_work: Optional[str]
    success_criteria_met: bool
    number_of_reviews: int=0

In [ ]:
# Step 2: Start the Graph Builder with this State class
graph_builder = StateGraph(State)

### Detailed Agent Setup
In this case, we are going to define two agents/nodes with different roles.  The first one will be a "history" expert.  The purpose of this agent is that it can do research on historical events and create a detailed report that will be saved as a markdown file.  This will be given a consistent prompt and the user should only have to supply an event.  The second agent/node is an evaluator.  This evaluator will review the report and ensure provide feedback.  Only after it is statified with the results will the process end.  This will also use two different LLMs with the history export using OpenAI and the reviewer using Gemini.  Each of these agents could also be set up with different tool sets as well, however, in this case, I will make the tools available to both.

#### Define the prompts
First define a detailed prompt for both agents.  These may very well need to be fined tuned to get them right.  You can use AI to get a jump start on crafting a good system prompt.

In [ ]:
history_expert_system_message = """
You are HistoryExpert, an autonomous research agent specializing in historical events. You have access to tools including Google Search, Wikipedia, and file management (read_file, write_file, list_directory, delete_file). Your mission is to take a single historical event as input and generate a deep, comprehensive, meticulously accurate historical report about that event, and SAVE IT TO DISK.

OUTPUT REQUIREMENTS:

1. Begin with a concise executive summary. This summary must contain 3–6 bullet points capturing the essential who, what, when, where, why, and outcome of the event. It must be clearly labeled with the header: "## Executive Summary".

2. After the summary, produce a detailed, lengthy, deeply researched historical report. Use section headers where appropriate. The report must include background, causes, chronology, major participants, significance, geopolitical impact, cultural or social consequences, and primary sources when available. Cite sources (Wikipedia or Google) in plain text, not with citation IDs or URLs unless required.

3. IMPORTANT - WRITE THE REPORT TO DISK: After creating the report content, you MUST use the write_file tool to save it. The filename should follow this exact format:
   FILENAME: YYYYMMDDHHmm_<event_name>.md
   Where YYYYMMDDHHmm is the current UTC timestamp (use today's date), and <event_name> is the event name in snake_case with no spaces.
   
   Example: If the event is "Battle of Hastings" today is 2024-11-15 at 14:30 UTC, the file should be saved as:
   "20241115143000_battle_of_hastings.md"

4. Tone requirements: Maintain an authoritative, neutral, academic, objective tone. Note when historians disagree. Avoid speculation unless it is explicitly labeled as speculation.

BEHAVIOR RULES:
- Prefer primary historical sources first, then Wikipedia, then Google.
- If the event name is ambiguous, identify any possible interpretations and ask for clarification if absolutely necessary.
- The report must always be highly detailed unless explicitly instructed otherwise.
- Do not produce a summary at the bottom; only at the top.
- ALWAYS save the file using the write_file tool when done - this is mandatory.

FINAL WORKFLOW:
1. Research the event using search and wikipedia tools if needed
2. Create the full report content (Executive Summary + detailed report)
3. Use write_file tool to save the report with the properly formatted filename
4. Confirm the file has been saved
"""

evaluator_system_message = """
You are HistoryReviewAgent, an autonomous quality-control reviewer responsible for evaluating the output generated by the HistoryExpert agent. 
You must evaluate the historical report strictly.  You are tough but fair, however, you always send initial constructive feedback, typically 
asking for expanded details.

Your responsibilities:

1. Evaluate whether the report fully complies with the required structure:
   - Contains an "## Executive Summary" at the top with 3–6 bullet points.
   - Contains a lengthy, detailed historical report covering:
       background, causes, chronology, major participants, significance,
       geopolitical impact, cultural/social consequences, and sources.
   - Uses section headers appropriately.
   - Maintains an objective, academic, historically accurate tone.
   - Avoids speculation unless explicitly labeled as such.
   - Has been saved to disk as a file (use read_file tool to verify the file exists in the sandbox directory).

2. If ALL success criteria are met:
   - Set success_criteria_met = true
   - Set feedback to a short confirmation message such as:
       "Report meets all criteria and has been saved to disk."
   - You should NOT update the number_of_reviews, except to check if it is equal to 0 (i.e. the first review) 

3. If ANY criteria are NOT met:
   - Set success_criteria_met = false
   - Set feedback to clear, constructive, actionable correction notes that the HistoryExpert can use.
   - You should NOT update the number_of_reviews.

IMPORTANT:
- You will ALWAYS success_criteria_met = false on the first 3 reviews (i.e. number_of_reviews <= 3) and ask for expanded details.
- The evaluator MUST NOT rewrite or fix the report — only evaluate it.
- The evaluator MUST include a file name in its response.
- Use list_directory and read_file tools to verify the file was actually written to disk.
- The only output should be the structured fields required by EvaluatorOutput.
"""


### Create the History Expert
This is similiar to the previous example, but gets a little more complicated on handling the different types of messages

In [ ]:
# Create the history expert

open_ai = ChatOpenAI(model="gpt-4o-mini")
tools_for_history_expert = get_tools()
open_ai_with_tools = open_ai.bind_tools(tools_for_history_expert)

def history_expert(state: State):
    # Convert incoming messages to BaseMessage objects
    messages = []
    for msg in state["messages"]:
        if isinstance(msg, dict):
            if msg["role"] == "system":
                messages.append(SystemMessage(content=msg["content"]))
            elif msg["role"] == "user":
                messages.append(HumanMessage(content=msg["content"]))
            elif msg["role"] == "assistant":
                messages.append(AIMessage(content=msg.get("content", ""), tool_calls=msg.get("tool_calls")))
            elif msg["role"] == "tool":
                messages.append(ToolMessage(content=msg["content"], tool_call_id=msg.get("tool_call_id")))
        else:
            messages.append(msg)
    # Ensure system message is present
    has_system_message = any(isinstance(m, SystemMessage) for m in messages)
    if not has_system_message:
        messages.insert(0, SystemMessage(content=history_expert_system_message))
    results = open_ai_with_tools.invoke(messages)
    return {"messages": [results]}


# Add history_expert and tools_for_history_expert nodes
graph_builder.add_node("history_expert", history_expert)
graph_builder.add_node("tools_for_history_expert", ToolNode(tools=tools_for_history_expert))

#### Structured Output
For the evaluator, we will define the structure that should be returned.  This will make routing easier.

In [ ]:
# Create the evaluator with structured output
class EvaluatorOutput(BaseModel):
    feedback: str = Field(description="Feedback on the assistant's response")
    success_criteria_met: bool = Field(description="Whether the success criteria have been met")
    number_of_reviews: int = Field(description="The number of reviews that have been conducted.")   

tools_for_evaluator = get_tools()
gemini = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=os.environ["GOOGLE_API_KEY"]
)
gemini_with_tools = gemini.bind_tools(tools_for_evaluator)
gemini_structured_with_tools = gemini_with_tools.with_structured_output(EvaluatorOutput)


def evaluator(state: State):
    # Convert incoming messages to BaseMessage objects
    messages = []
    for msg in state["messages"]:
        if isinstance(msg, dict):
            if msg["role"] == "system":
                messages.append(SystemMessage(content=msg["content"]))
            elif msg["role"] == "user":
                messages.append(HumanMessage(content=msg["content"]))
            elif msg["role"] == "assistant":
                messages.append(AIMessage(content=msg.get("content", ""), tool_calls=msg.get("tool_calls")))
            elif msg["role"] == "tool":
                messages.append(ToolMessage(content=msg["content"], tool_call_id=msg.get("tool_call_id")))
        else:
            messages.append(msg)
    # Ensure system message is present
    has_system_message = any(isinstance(m, SystemMessage) for m in messages)
    if not has_system_message:
        messages.insert(0, SystemMessage(content=evaluator_system_message))

    results = gemini_structured_with_tools.invoke(messages)

    new_state = {
        "messages": [
            {
                "role": "assistant",
                "content": f"Evaluator Feedback on this answer: {results.feedback}",
            }
        ],
        "feedback_on_work": results.feedback,
        "success_criteria_met": results.success_criteria_met,
        "number_of_reviews": results.number_of_reviews + 1,
    }
    return new_state


# Add evaluator and tools_for_evaluator nodes
graph_builder.add_node("evaluator", evaluator)
graph_builder.add_node("tools_for_evaluator", ToolNode(tools=tools_for_evaluator))

### Define condition statements.
In this case, will not just use the out of the box tool router as we have multiple different tools calls and some more complex flows 

In [ ]:
MAX_NUMBER_OF_REVIEWS = 10

def route_history_expert(state: State):
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools_for_history_expert"
    return "evaluator"


def route_evaluator(state: State):
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools_for_evaluator"
    if state["success_criteria_met"] or state["number_of_reviews"] >= MAX_NUMBER_OF_REVIEWS:
        return "END"
    else:
        return "history_expert"

In [ ]:
# Step 4: Create the edges

# Entry point
graph_builder.add_edge(START, "history_expert")

# From history_expert: if needs tools, go to tools; otherwise go to evaluator
graph_builder.add_conditional_edges(
    "history_expert", 
    route_history_expert, 
    {"tools_for_history_expert": "tools_for_history_expert", "evaluator": "evaluator"}
)

# From tools_for_history_expert: always back to history_expert to process tool results
graph_builder.add_edge("tools_for_history_expert", "history_expert")

# From evaluator: if needs tools, go to tools; otherwise check if done or needs to go back to the history expert
graph_builder.add_conditional_edges(
    "evaluator", 
    route_evaluator, 
    {"tools_for_evaluator": "tools_for_evaluator", "history_expert": "history_expert", "END": END}
)

# From tools_for_evaluator: always back to evaluator to process tool results  
graph_builder.add_edge("tools_for_evaluator", "evaluator")

In [ ]:
# Step 5: Compile the graph
graph = graph_builder.compile(checkpointer=sql_memory)
display(Image(graph.get_graph().draw_mermaid_png()))

### 5. Test it Out
Use Gradio to create a front end that will allow us to test this out

In [ ]:
memory_key = "jay_123"

config = {"configurable": {"thread_id": uuid7()}}

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gradio.ChatInterface(chat, type="messages").launch()